In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
from pettingzoo.classic import connect_four_v3
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"
from IPython.display import clear_output
import random

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-wfqpdj_k because the default path (/home/epic_euclid/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
#Dimensions du tableau
height=6
width=7

#Fonction qui initialise la matrice Q
def initialize_Q(): #Initialise la matrice Q (que des zéros)
    return np.zeros((height, width, width)) 
def find_best_legal(arr, mask):
    legal_gains=arr*mask
    best_action=np.argmax(legal_gains)
    if legal_gains[best_action] != 0:
        return best_action
    else :
        action=random.randint(0,6)
        while mask[action]==0:
            action=random.randint(0,6)
        return action
def left_legal(actions,mask):
    for i in range(len(actions)):
        if not (mask[i]==0) :
            return i

class Player:
    def __init__(self, _id, Q, epsilon=0.7, rng=None): 
        self.name = 'Player '+str(_id)
        self.Q=initialize_Q()
        self.st=(None,None) #Etat
        self._id=_id #Identifiant
        self.epsilon=epsilon
        self.left=False
        
    def get_action(self, obs, epsilon=None):
        mask=obs["action_mask"]
        if self.left:
            actions=[0,1,2,3,4,5,6]
            return left_legal(actions,mask)
        if (random.uniform(0,1)<epsilon) or (self.st == (None,None)): #Exploration ou premier coupde la partie
            action=random.randint(0,6)
            while mask[action]==0:
                action=random.randint(0,6)
        else:
            (x,y)=self.st
            Q=self.Q
            #print(x,y)
            action=find_best_legal(Q[x,y], mask) #On choisit la meilleure action parmi celles qu'il peut jouer (meilleure dans le sens maximise Q)
        return action
    
    def highest_not_null(self, action, obs, terminated): #Fonction auxiliaire à la fonction pour déterminer l'état
        if not terminated:
            idx=1
        else:
            idx=0
        for i in range(height):
            if obs["observation"][:, action, idx][i] != 0: 
                return i

    def update_state(self,action, obs, terminated): #Fonction qui update l'état du joueur (trouve en fait le dernier coup joué)
        y=action
        x=self.highest_not_null(action, obs, terminated)
        self.st=(x,y)
    def update_Q(self, last_state, obs, action, r, alpha, gamma,show=False): #fonction qui update Q selon 
        next_action=self.get_action(obs, epsilon=0.0)
        (a,b)=self.st
        (x,y)=last_state
        Q=self.Q
        pic=Q[x,y] #Garder en mémoire les anciennes récompenses pour chaque action de l'état (x,y) (pour la visualisation)
        self.Q[x,y][action]=(1-alpha)*Q[x,y][action] + alpha*(r + gamma*Q[a,b][next_action])#-Q[x,y][action]) #modif update_Q
        if show:
            print("Pour l'état ( "+str(x)+" , "+str(y)+" ), voici les anciennes récompenses : "+str(pic)+" et voici les nouvelles : "+str(self.Q[x,y]))

def play_game(env, agent0, agent1, alpha, gamma, display=False): #e epsilon coeff d'exploration
    done = False
    env.reset()
    obs, _, _, _, _ = env.last()
    l=[agent0, agent1]
    random.shuffle(l)
    while not done:
        #print("")
        #print("")
        for i, agent in enumerate(l):
            last_state=agent.st
            e=agent.epsilon
            action = agent.get_action(obs, epsilon=e) #Epsilon = taux d'exploration #Il s'agit de obs du point de vue de l'autre joueur mais on veut que le mask
            env.step(action) #Préciser à l'environnement pettingZoo que l'action prise
            obs, reward, terminated, _, _ = env.last()
            agent.update_state(action, obs, terminated) #Mise à jour de l'état de l'agent

            if display:
                clear_output(wait=True)
                plt.imshow(env.render())
                plt.show()
            done = terminated #Déroulement fini
            if np.sum(obs["action_mask"]) == 0: #Vérification d'une éventuelle égalité
                if display: 
                    print('Draw')
                done=True
                break
                
            if last_state != (None, None) : #Mise à jour des récompenses dans Q
                agent.update_Q(last_state, obs, action, reward, alpha, gamma, show=False)
            
            if done: #Victoire d'un des joueurs
                if display:
                    print(f"{agent.name} : won")
                    print(obs['observation'][:, :, 0]- obs['observation'][:, :, 1])
                    print('last_state : ', last_state)
                    print('action : ', action)
                done=True
                break       
            

In [3]:
def play_game_count(env, agent0, agent1, alpha, gamma, display=False, dic={}): #e epsilon coeff d'exploration
    done = False
    env.reset()
    obs, _, _, _, _ = env.last()
    l=[agent0, agent1]
    random.shuffle(l)
    while not done:
        #print("")
        #print("")
        for i, agent in enumerate(l):
            #print("")
            #print("Player : "+str(agent._id))
                            #print(type(agent))
            e=agent.epsilon
            last_state=agent.st
            #print('last_state : ',last_state)
            action = agent.get_action(obs, epsilon=e) #Epsilon = taux d'exploration
            #print('action : ',action)
            #print(action)
            env.step(action) #Préciser à l'environnement pettingZoo que l'action prise
            obs, reward, terminated, _, _ = env.last()
            #print("obs : ",obs['observation'][:, :, 1])
            agent.update_state(action, obs, terminated) #Mise à jour de l'état de l'agent

            if display:
                clear_output(wait=True)
                plt.imshow(env.render())
                plt.show()
            done = terminated #Déroulement fini
            if np.sum(obs["action_mask"]) == 0: #Vérification d'une éventuelle égalité
                if display: 
                    print('Draw')
                done=True
                result='Draw'
                break

            #if last_state != (None, None) : #Mise à jour des récompenses dans Q #Pas de mise à jour des récompenses pour l'évaluation
                #agent.update_Q(last_state, obs, action, reward, alpha, gamma, show=False)

            if done: #Victoire d'un des joueurs
                if display:
                    print(f"{agent.name} : won")
                    print(obs['observation'][:, :, 0]- obs['observation'][:, :, 1])
                    print('last_state : ', last_state)
                    print('action : ', action)
                result=agent.name
                break
    dic[result]+=1
                

In [4]:
def multiple_games_count(env, agent0, agent1, alpha, gamma , count):
    dic={}
    dic['Draw']=0
    dic[agent0.name]=0
    dic[agent1.name]=0
    for i in tqdm(range(count)):
        play_game_count(env, agent0, agent1, alpha, gamma, display=False, dic=dic)
    return dic

Entrainement et Evaluation

In [5]:
env = connect_four_v3.env(render_mode="rgb_array")
env.reset()
agent0=Player(_id=1, Q=initialize_Q(), epsilon=0.3) #Il explore 30% du tps
random_player=Player(_id=2, Q=initialize_Q(), epsilon =1) #Random explore 100% du tps
epsilon_training_mate=1
training_mate=Player(_id=2, Q=initialize_Q(), epsilon =epsilon_training_mate)
alpha=0.01
gamma=0.5
loop=10000

In [7]:
for i in tqdm(range(2*loop)): #Entraînement
    play_game(env, agent0, training_mate, alpha, gamma , display=False)

100%|██████████| 20000/20000 [00:55<00:00, 357.95it/s]


In [9]:
agent0.epsilon=0 #On le passe en mode exploitation
random_player.left=False
multiple_games_count(env, agent0, random_player, alpha, gamma , loop)    

100%|██████████| 10000/10000 [00:18<00:00, 545.29it/s]


{'Draw': 0, 'Player 1': 8595, 'Player 2': 1405}

Parameters Choice

In [37]:
dic_={}
for alpha in [0.001, 0.01, 0.1]:
    for gamma in [0.5,0.9]:
        print("Processing : ",alpha,gamma)
        env = connect_four_v3.env(render_mode="rgb_array")
        env.reset()
        agent0=Player(_id=1, Q=initialize_Q(), epsilon=0.3)
        training_mate=Player(_id=2, Q=initialize_Q(), epsilon=1)
        agent_random=Player(_id="Random", Q=initialize_Q(), epsilon=1)
        e=0.0
        loop=10000
        for i in tqdm(range(loop)): #training
            play_game(env, agent0, training_mate, alpha, gamma , display=False)
        dic=multiple_games_count(env, agent0, random_player, alpha, gamma , loop) 
        dic_[(alpha,gamma)]=dic
dic_

Processing :  0.001 0.5


100%|██████████| 10000/10000 [00:22<00:00, 450.41it/s]


Processing :  0.001 0.9


100%|██████████| 10000/10000 [00:22<00:00, 442.08it/s]


Processing :  0.1 0.5


...

Processing :  0.1 0.5


100%|██████████| 10000/10000 [00:18<00:00, 547.55it/s]


Processing :  0.1 0.9


100%|██████████| 10000/10000 [00:18<00:00, 533.37it/s]

{(0.001, 0.5): {'Draw': 7, 'Player 1': 6148, 'Player 2': 3845}, (0.001, 0.9): {'Draw': 18, 'Player 1': 5926, 'Player 2': 4056}, (0.1, 0.5): {'Draw': 11, 'Player 1': 7654, 'Player 2': 2335}, (0.1, 0.9): {'Draw': 8, 'Player 1': 7418, 'Player 2': 2574}}
